In [1]:
# Ignoring the warnings
import warnings
warnings.filterwarnings('ignore')

# Imorting required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Importing require AI/ML Libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier as xgb
from sklearn.preprocessing import StandardScaler


In [2]:
# Location of the datasets
train_data=r'C:\Users\DELL\train_depression.csv'
test_data=r'C:\Users\DELL\test_depression.csv'

In [3]:
# Locading the datasets in memory
df_train=pd.read_csv(train_data)

### Missing Value Imputation

In [4]:
# Checking the % of null entries
df_train.isnull().sum()/len(df_train)*100

id                                        0.000000
Name                                      0.000000
Gender                                    0.000000
Age                                       0.000000
City                                      0.000000
Working Professional or Student           0.000000
Profession                               26.034115
Academic Pressure                        80.172708
Work Pressure                            19.842217
CGPA                                     80.171997
Study Satisfaction                       80.172708
Job Satisfaction                         19.836532
Sleep Duration                            0.000000
Dietary Habits                            0.002843
Degree                                    0.001421
Have you ever had suicidal thoughts ?     0.000000
Work/Study Hours                          0.000000
Financial Stress                          0.002843
Family History of Mental Illness          0.000000
Depression                     

In [5]:
# It is observed that certain columns have very low missing value count like Financial Stress , Degree & Dietary Habits
# So here the missing value imputation can be done by using straight forward methods like mean , median or mode 
# For that it is necessary to find out their data types
df_train.dtypes

id                                         int64
Name                                      object
Gender                                    object
Age                                      float64
City                                      object
Working Professional or Student           object
Profession                                object
Academic Pressure                        float64
Work Pressure                            float64
CGPA                                     float64
Study Satisfaction                       float64
Job Satisfaction                         float64
Sleep Duration                            object
Dietary Habits                            object
Degree                                    object
Have you ever had suicidal thoughts ?     object
Work/Study Hours                         float64
Financial Stress                         float64
Family History of Mental Illness          object
Depression                                 int64
dtype: object

In [6]:
# Checking the Financial stress parameter --> It is clear that this column needs to be treated as a categorical variable
df_train['Financial Stress'].unique()

array([ 2.,  3.,  1.,  4.,  5., nan])

In [7]:
# Changing the data type of the Financial Stress to object
df_train['Financial Stress']=df_train['Financial Stress'].astype(object)

In [8]:
# Extracting the columns which have null values along with their data type
null_value_cols=[(col,round(100*null_values/len(df_train),2),df_train[col].dtype)
                 for col,null_values in df_train.isnull().sum().items() if null_values>0]

In [9]:
# Checking the null values along with their data types
null_value_cols

[('Profession', 26.03, dtype('O')),
 ('Academic Pressure', 80.17, dtype('float64')),
 ('Work Pressure', 19.84, dtype('float64')),
 ('CGPA', 80.17, dtype('float64')),
 ('Study Satisfaction', 80.17, dtype('float64')),
 ('Job Satisfaction', 19.84, dtype('float64')),
 ('Dietary Habits', 0.0, dtype('O')),
 ('Degree', 0.0, dtype('O')),
 ('Financial Stress', 0.0, dtype('O'))]

In [10]:
# I will extract the columns which have very low missing values
low_missing_values=[(col,dt) for col,missing_pct,dt in null_value_cols if missing_pct<1]

In [11]:
# Checking columns which have low missing value count and their dtypes
low_missing_values

[('Dietary Habits', dtype('O')),
 ('Degree', dtype('O')),
 ('Financial Stress', dtype('O'))]

In [12]:
# Imputing the missing values in the columns (with low missing values) and which are categorical in nature by using mode
for each_col,data_type in low_missing_values:
  df_train[each_col]=df_train[each_col].fillna(df_train[each_col].mode()[0])

In [13]:
# Checking the % of null entries
df_train.isnull().sum()/len(df_train)*100

id                                        0.000000
Name                                      0.000000
Gender                                    0.000000
Age                                       0.000000
City                                      0.000000
Working Professional or Student           0.000000
Profession                               26.034115
Academic Pressure                        80.172708
Work Pressure                            19.842217
CGPA                                     80.171997
Study Satisfaction                       80.172708
Job Satisfaction                         19.836532
Sleep Duration                            0.000000
Dietary Habits                            0.000000
Degree                                    0.000000
Have you ever had suicidal thoughts ?     0.000000
Work/Study Hours                          0.000000
Financial Stress                          0.000000
Family History of Mental Illness          0.000000
Depression                     

In [14]:
# I decided to write a function to identify the columns which very high percentage of missing values and then just imputing missing values by a place holder 
# But before going ahead I decided to check the nature of the columns with these high missing values
high_missing_values=[(col,dt) for col,missing_pct,dt in null_value_cols if missing_pct>50]
high_missing_values

[('Academic Pressure', dtype('float64')),
 ('CGPA', dtype('float64')),
 ('Study Satisfaction', dtype('float64'))]

In [15]:
#Just extracting the column names of the columns with high missing values
col_names_high_missing_value=[each_col for each_col,data_type in high_missing_values]

# Doing a quick analysis of the columns with high missing values 
def analyze_high_missing_value_columns(dataframe,high_missing_value_col):

  """
  This function will take data frame & the columns with high missing values.
    Then it will show the number of unique values in column with some statistical information of the selected column
    
    Args: Dataframe & columns with high missing values
    
    Returns : Unique values in the columns and statistics for the column
  """

  for each_col in high_missing_value_col:
    print(f'For {each_col}')
    print(f'The number of unique values are {dataframe[each_col].nunique()}' )
    print(f'The statistics are {dataframe[each_col].describe()} \n' )


In [16]:
# Using the modular function to check the details for all columns with high missing values
analyze_high_missing_value_columns(df_train,col_names_high_missing_value)

For Academic Pressure
The number of unique values are 5
The statistics are count    27897.000000
mean         3.142273
std          1.380457
min          1.000000
25%          2.000000
50%          3.000000
75%          4.000000
max          5.000000
Name: Academic Pressure, dtype: float64 

For CGPA
The number of unique values are 331
The statistics are count    27898.000000
mean         7.658636
std          1.464466
min          5.030000
25%          6.290000
50%          7.770000
75%          8.920000
max         10.000000
Name: CGPA, dtype: float64 

For Study Satisfaction
The number of unique values are 5
The statistics are count    27897.000000
mean         2.944940
std          1.360197
min          1.000000
25%          2.000000
50%          3.000000
75%          4.000000
max          5.000000
Name: Study Satisfaction, dtype: float64 



In [17]:
# Making small bins of the CGPA 
def CGPA_BIN(CGPA_VALUE):
  if CGPA_VALUE<5:
    return "Less than 5"
  elif CGPA_VALUE>=5 and CGPA_VALUE<=6:
    return "5-6"
  elif CGPA_VALUE>=6 and CGPA_VALUE<=7:
    return "6-7"
  elif CGPA_VALUE>=7 and CGPA_VALUE<=8:
    return "7-8"
  elif CGPA_VALUE>=8 and CGPA_VALUE<=9:
    return "8-9"
  elif CGPA_VALUE>=9 and CGPA_VALUE<=10:
    return "9-10"  

In [18]:
# Replacing the CGPA column
df_train.CGPA=df_train.CGPA.apply(lambda x : CGPA_BIN(x))